In [2]:
import numpy as np
import random

In [3]:
class AntColony:
    def __init__(self, distance_matrix, num_ants, num_iterations, decay, alpha, beta):
        self.distance_matrix = distance_matrix
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta
        self.num_cities = len(distance_matrix)
        self.pheromone_matrix = np.ones((self.num_cities, self.num_cities))
        self.best_solution = None
        self.best_distance = np.inf
    
    def run(self):
        for i in range(self.num_iterations):
            solutions = []
            for j in range(self.num_ants):
                solution = self.build_solution()
                solutions.append(solution)
                distance = self.evaluate_solution(solution)
                if distance < self.best_distance:
                    self.best_solution = solution
                    self.best_distance = distance
            self.update_pheromone_matrix(solutions)
            
    def build_solution(self):
        unvisited_cities = set(range(self.num_cities))
        current_city = random.choice(list(unvisited_cities))
        unvisited_cities.remove(current_city)
        solution = [current_city]
        while unvisited_cities:
            probabilities = self.calculate_probabilities(current_city, unvisited_cities)
            next_city = self.select_next_city(probabilities, list(unvisited_cities))
            solution.append(next_city)
            unvisited_cities.remove(next_city)
            current_city = next_city
        return solution
    
    def calculate_probabilities(self, current_city, unvisited_cities):
        pheromone_values = self.pheromone_matrix[current_city][list(unvisited_cities)]
        distances = self.distance_matrix[current_city][list(unvisited_cities)]
        attractiveness_values = 1 / distances ** self.beta
        probabilities = pheromone_values ** self.alpha * attractiveness_values
        probabilities /= probabilities.sum() 
        return probabilities
    
    def select_next_city(self, probabilities, unvisited_cities):
        index = np.random.choice(range(len(unvisited_cities)), p=probabilities)
        return unvisited_cities[index]
    
    def evaluate_solution(self, solution):
        distance = 0
        for i in range(self.num_cities - 1):
            city1 = solution[i]
            city2 = solution[i + 1]
            distance += self.distance_matrix[city1][city2]
        distance += self.distance_matrix[solution[-1]][solution[0]]
        return distance
    
    def update_pheromone_matrix(self, solutions):
        self.pheromone_matrix *= self.decay
        for solution in solutions:
            distance = self.evaluate_solution(solution)
            for i in range(self.num_cities - 1):
                city1 = solution[i]
                city2 = solution[i + 1]
                self.pheromone_matrix[city1][city2] += 1 / distance
            self.pheromone_matrix[solution[-1]][solution[0]] += 1 / distance



In [6]:
# Define the distance matrix
distance_matrix = np.array([
    [0, 3, 6, 2, 3],
    [3, 0, 5, 2, 3],
    [6, 5, 0, 6, 4],
    [2, 2, 6, 0, 6],
    [3, 3, 4, 6, 0]
])

# Set the ACO parameters
num_ants = 50
num_iterations = 2
decay = 0.5
alpha = 1
beta = 2

# Create an instance of the AntColony class
aco = AntColony(distance_matrix, num_ants, num_iterations, decay, alpha, beta)

# Run the algorithm
aco.run()

# Print the best solution and its distance
print("Best solution:", aco.best_solution)
print("Best distance:", aco.best_distance)

Best solution: [1, 3, 0, 4, 2]
Best distance: 16
